In [ ]:
!pip install transformers==3.0.2

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1.tar.gz (97 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [ ]:

# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Specify your new header names here
new_header_content = ['TweetID', 'Tweet']  # Add or modify column names as needed

# Read the TSV file without an existing header and add the new header
tweet_content = pd.read_csv('/content/drive/MyDrive/SMM4H-Task1/Train_2024/tweets.tsv', sep='\t', header=None, names=new_header_content)

# Display the DataFrame to verify the new header
tweet_content.head()

,TweetID,Tweet
0,SMM4H2022yW4mXhmCE9gh1B7b,"do you have any medication allergies? ""asthma!..."
1,SMM4H2022afMxPjtHKyItF0n3,"@USER_________ if #avelox has hurt your liver,..."
2,SMM4H2022dc1lWHnZeBy441B8,"apparently, baclofen greatly exacerbates the ""..."
3,SMM4H2022ExV2GXXaqhGjJNFM,pt of mine died from cipro rt @USER_________: ...
4,SMM4H2022dZ7ZqyvGqxh0jKeH,@USER_____ The only pt of mine who ever died w...


In [ ]:
# Specify your new header names here
new_header_label = ['TweetID', 'ADR', 'start','end','texts','MEDDRA']  # Add or modify column names as needed

# Read the TSV file without an existing header and add the new header
tweet_label = pd.read_csv('/content/drive/MyDrive/SMM4H-Task1/Train_2024/gold_annotations_complete.tsv', sep='\t', header=None, names=new_header_label)

# Display the DataFrame to verify the new header
tweet_label.head()

,TweetID,ADR,start,end,texts,MEDDRA
0,SMM4H2022yW4mXhmCE9gh1B7b,ADE,28,37,allergies,10013661
1,SMM4H2022afMxPjtHKyItF0n3,ADE,31,46,HURT YOUR Liver,10024668
2,SMM4H2022dc1lWHnZeBy441B8,ADE,48,50,AD,10003731
3,SMM4H2022dc1lWHnZeBy441B8,ADE,88,93,focus,10003738
4,SMM4H2022ExV2GXXaqhGjJNFM,ADE,11,15,died,10011906


In [ ]:
import re
def clean_text(text: str) -> str:
    # Remove HTML/Markdown tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove @user mentions
    text = re.sub(r'@\w+', '', text)
    # Convert hashtags into words by removing the # but keeping the word
    text = re.sub(r'#(\w+)', r'\1', text)
    # Remove special non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Remove any remaining special characters (punctuation, etc.)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove underscores
    text = text.replace('_', '')
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove leading and trailing spaces
    text = text.strip()
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)  # Keeps only letters, numbers, and whitespace
    return text
tweet_content['Tweet'] = tweet_content['Tweet'].apply(clean_text)
tweet_content

,TweetID,Tweet
0,SMM4H2022yW4mXhmCE9gh1B7b,do you have any medication allergies asthma me...
1,SMM4H2022afMxPjtHKyItF0n3,if avelox has hurt your liver avoid tylenol al...
2,SMM4H2022dc1lWHnZeBy441B8,apparently baclofen greatly exacerbates the ad...
3,SMM4H2022ExV2GXXaqhGjJNFM,pt of mine died from cipro rt if only more doc...
4,SMM4H2022dZ7ZqyvGqxh0jKeH,the only pt of mine who ever died was one age ...
...,...,...
17969,SMM4H2024bYicTprxagHNGgs,tamec
17970,SMM4H2024z5V2WTPOV9RxE41,thinking about buying nolvadex tablets hubei t...
17971,SMM4H2024E18a1sHgYoqpmlX,sandpiper trial presented at asco18 addition o...
17972,SMM4H20249XBQmnyF0rsHyHt,my sestina luna taxus baccata after sylviaplat...


In [ ]:
# task 1: determine the ADE existence
# Perform a left join on 'tweetID'
task1data = pd.merge(tweet_content, tweet_label[['TweetID', 'ADR']], on='TweetID', how='left')
# Display the DataFrame to verify the merge
task1data

,TweetID,Tweet,ADR
0,SMM4H2022yW4mXhmCE9gh1B7b,do you have any medication allergies asthma me...,ADE
1,SMM4H2022afMxPjtHKyItF0n3,if avelox has hurt your liver avoid tylenol al...,ADE
2,SMM4H2022dc1lWHnZeBy441B8,apparently baclofen greatly exacerbates the ad...,ADE
3,SMM4H2022dc1lWHnZeBy441B8,apparently baclofen greatly exacerbates the ad...,ADE
4,SMM4H2022ExV2GXXaqhGjJNFM,pt of mine died from cipro rt if only more doc...,ADE
...,...,...,...
18440,SMM4H2024bYicTprxagHNGgs,tamec,NaN
18441,SMM4H2024z5V2WTPOV9RxE41,thinking about buying nolvadex tablets hubei t...,NaN
18442,SMM4H2024E18a1sHgYoqpmlX,sandpiper trial presented at asco18 addition o...,NaN
18443,SMM4H20249XBQmnyF0rsHyHt,my sestina luna taxus baccata after sylviaplat...,NaN


In [ ]:
task1data['Label'] = np.where(task1data['ADR'].isna(), 0, 1)
new_df = task1data[['Tweet', 'Label']]
new_df

,Tweet,Label
0,do you have any medication allergies asthma me...,1
1,if avelox has hurt your liver avoid tylenol al...,1
2,apparently baclofen greatly exacerbates the ad...,1
3,apparently baclofen greatly exacerbates the ad...,1
4,pt of mine died from cipro rt if only more doc...,1
...,...,...
18440,tamec,0
18441,thinking about buying nolvadex tablets hubei t...,0
18442,sandpiper trial presented at asco18 addition o...,0
18443,my sestina luna taxus baccata after sylviaplat...,0


In [ ]:
# Read the TSV file without an existing header and add the new header
tweet_content_test = pd.read_csv('/content/drive/MyDrive/SMM4H-Task1/Dev_2024/tweets.tsv', sep='\t', header=None, names=new_header_content)
# Display the DataFrame to verify the new header
tweet_content_test.head()
# Read the TSV file without an existing header and add the new header
tweet_label_test = pd.read_csv('/content/drive/MyDrive/SMM4H-Task1/Dev_2024/gold_annotations_complete.tsv', sep='\t', header=None, names=new_header_label)
# Display the DataFrame to verify the new header
tweet_label_test.head()
tweet_content_test['Tweet'] = tweet_content_test['Tweet'].apply(clean_text)
# Perform a left join on 'tweetID'
task1data_test = pd.merge(tweet_content_test, tweet_label_test[['TweetID', 'ADR']], on='TweetID', how='left')
# Display the DataFrame to verify the merge
task1data_test['Label'] = np.where(task1data_test['ADR'].isna(), 0, 1)
new_df_test = task1data_test[['Tweet', 'Label']]
new_df_test.drop_duplicates(inplace=True)
new_df_test.reset_index(drop=True, inplace=True)
new_df_test = new_df_test.sample(frac=1, random_state=42).reset_index(drop=True)
new_df_test

<ipython-input-26-e3e5f496d7b2>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_test.drop_duplicates(inplace=True)


,Tweet,Label
0,when you find out you should be prescribed wit...,0
1,things no 19 year old should be excited about ...,0
2,there is a growing trend toward use of combina...,0
3,latuda will cure your manic depressive disorde...,0
4,i got an symbicort inhaler probiotics fish oil...,0
...,...,...
927,t3 many drugs eg cymbalta already approved 4 u...,0
928,its been about 2 weeks since i coldturkey quit...,0
929,the painkiller vioxx kills people as many as a...,0
930,i hate yooouuuu humira crohnspictwittercomgwsr...,0


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Tweet
        self.targets = self.data.Label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 1
# train_data=new_df.sample(frac=train_size,random_state=200)
train_data = new_df
# test_data=new_df.drop(train_data.index).reset_index(drop=True)
test_data = new_df_test
# train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (18445, 2)
TRAIN Dataset: (18445, 2)
TEST Dataset: (932, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **train_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
EPOCHS = 8
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 1.61995267868042
Training Accuracy per 5000 steps: 12.5


1153it [04:04,  4.72it/s]


The Total Accuracy for Epoch 0: 92.89780428300352
Training Loss Epoch: 0.21002701901348514
Training Accuracy Epoch: 92.89780428300352


1it [00:00,  5.48it/s]

...

The Total Accuracy for Epoch 6: 99.4090539441583
Training Loss Epoch: 0.020180283648607904
Training Accuracy Epoch: 99.4090539441583


1it [00:00,  5.45it/s]

Training Loss per 5000 steps: 0.004159130156040192
Training Accuracy per 5000 steps: 100.0


1153it [04:03,  4.73it/s]

The Total Accuracy for Epoch 7: 99.58796421794524
Training Loss Epoch: 0.014320830678543714
Training Accuracy Epoch: 99.58796421794524


In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

1it [00:00,  7.21it/s]

Validation Loss per 100 steps: 0.006476566661149263
Validation Accuracy per 100 steps: 100.0


59it [00:04, 13.54it/s]

Validation Loss Epoch: 0.17779581160647143
Validation Accuracy Epoch: 95.49356223175965
Accuracy on test data = 95.49%


In [ ]:
import torch
from torch import nn
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def valid(model, testing_loader, device, loss_function):
    model.eval()
    all_predictions = []
    all_targets = []
    total_loss = 0

    with torch.no_grad():
        for step, data in tqdm(enumerate(testing_loader, 0), desc='Validating'):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs.data, dim=1)
            all_predictions.extend(predicted_labels.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

            if step % 5000 == 0:
                current_loss = total_loss / (step + 1)
                print(f"Validation Loss per 100 steps: {current_loss}")

    # Convert all accumulated predictions and targets into a single array for metric calculation
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    # Calculate overall metrics
    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions, average='macro', zero_division=0)
    recall = recall_score(all_targets, all_predictions, average='macro', zero_division=0)
    f1 = f1_score(all_targets, all_predictions, average='macro', zero_division=0)

    # Calculate per-class metrics
    class_precision = precision_score(all_targets, all_predictions, average=None, zero_division=0)
    class_recall = recall_score(all_targets, all_predictions, average=None, zero_division=0)
    class_f1 = f1_score(all_targets, all_predictions, average=None, zero_division=0)

    epoch_loss = total_loss / len(testing_loader)
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {accuracy}")
    print(f"Validation Precision Epoch: {precision}")
    print(f"Validation Recall Epoch: {recall}")
    print(f"Validation F1 Score Epoch: {f1}")

    # Printing class-specific metrics
    for i, (cp, cr, cf) in enumerate(zip(class_precision, class_recall, class_f1)):
        print(f"Class {i}: Precision: {cp}, Recall: {cr}, F1: {cf}")

    return {
        'loss': epoch_loss,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'class_precision': class_precision,
        'class_recall': class_recall,
        'class_f1': class_f1
    }

In [ ]:
acc = valid(model, testing_loader, device, loss_function)

Validating: 2it [00:00, 13.77it/s]

Validation Loss per 100 steps: 0.00022691278718411922


Validating: 59it [00:04, 13.88it/s]

Validation Loss Epoch: 0.17780065544625898
Validation Accuracy Epoch: 0.9549356223175965
Validation Precision Epoch: 0.8173317775184753
Validation Recall Epoch: 0.8619288439357644
Validation F1 Score Epoch: 0.8378190255220417
Class 0: Precision: 0.9813302217036173, Recall: 0.9700115340253749, F1: 0.9756380510440835
Class 1: Precision: 0.6533333333333333, Recall: 0.7538461538461538, F1: 0.7


In [ ]:
output_model_file = 'pytorch_roberta_activitylabel.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


## testing to prediction on our test data

In [ ]:
prediction_data = pd.read_csv('/content/drive/MyDrive/SMM4H-Task1/tweets.tsv', delimiter='\t',encoding='latin1', names=new_header_content)
prediction_data
prediction_data['Tweet'] = prediction_data['Tweet'].apply(clean_text)
new_testing = prediction_data[['Tweet']]
prediction_data

,TweetID,Tweet
0,SMM4H2024MIGD1X8bP8fcNmo,im poppin the prozac im having a panic attack ...
1,SMM4H20244TFNHjiHoKCUHk5,can u plis manage viox i will pay
2,SMM4H2024DCpdwZ3mMPsXaZY,you may bring some ziprasidone for the instruc...
3,SMM4H2024QakWzPHeKAPeVnq,dark machinations involving propecia laid the ...
4,SMM4H2024j0ReC0r3UYUtnT0,ovarian cyst risk factors infertility treatmen...
...,...,...
11794,SMM4H2024GVCLpy343qmSeb6,tamiflu is my newest best friend
11795,SMM4H2024aijGWCqUlG2Ujnn,havent checked numbers but it appears 2011 pro...
11796,SMM4H2024pG2CrHmI7r6KAq1,cant forget the saltines i have to force down ...
11797,SMM4H2024m5tXtKyWIPOvFv0,i wasnt necessarily talking about people who s...


In [ ]:
import torch
from transformers import RobertaModel, RobertaTokenizer

# Load the fine-tuned model
model_path = 'pytorch_roberta_activitylabel.bin'
model = torch.load(model_path)

# Load the tokenizer
vocab_path = './'  # Assuming vocabulary files are saved here
tokenizer = RobertaTokenizer.from_pretrained(vocab_path)

In [ ]:
class SentimentData1(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Tweet
        #self.targets = self.data.Label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            #'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

new_test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

# Create the testing_set with has_labels=False
new_testing_set = SentimentData1(new_testing, tokenizer, MAX_LEN)

# Assuming new_test_params is defined as before
new_testing_loader = DataLoader(new_testing_set, **new_test_params)

In [ ]:
model_path = '/content/pytorch_roberta_activitylabel.bin'
model = torch.load(model_path, map_location=device)  # Ensure model is loaded to the correct device

def inference(model, testing_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            predictions.append(big_idx.cpu().numpy())

            if _ % 5000 == 0:
                print(f"Processed {_} steps.")
    predictions = np.concatenate(predictions, axis=0)

    return predictions

predictions = inference (model, new_testing_loader)
predictions[0:10]

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2it [00:00, 12.80it/s]

Processed 0 steps.


738it [00:54, 13.51it/s]


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# Assuming 'predictions' is your list of prediction results
prediction_data['Label'] = predictions
prediction_data_result = prediction_data[["TweetID","Label"]]
# Save the DataFrame to a TXT file, values separated by commas
prediction_data_result.to_csv('/content/prediction_data_results.csv', sep=',', index=False)

In [ ]:
merged_data = pd.merge(prediction_data, prediction_data_result, on='TweetID', how='left')
merged_data

,TweetID,Tweet,Label
0,SMM4H2024MIGD1X8bP8fcNmo,im poppin the prozac im having a panic attack ...,1
1,SMM4H20244TFNHjiHoKCUHk5,can u plis manage viox i will pay,0
2,SMM4H2024DCpdwZ3mMPsXaZY,you may bring some ziprasidone for the instruc...,0
3,SMM4H2024QakWzPHeKAPeVnq,dark machinations involving propecia laid the ...,0
4,SMM4H2024j0ReC0r3UYUtnT0,ovarian cyst risk factors infertility treatmen...,0
...,...,...,...
11794,SMM4H2024GVCLpy343qmSeb6,tamiflu is my newest best friend,0
11795,SMM4H2024aijGWCqUlG2Ujnn,havent checked numbers but it appears 2011 pro...,0
11796,SMM4H2024pG2CrHmI7r6KAq1,cant forget the saltines i have to force down ...,0
11797,SMM4H2024m5tXtKyWIPOvFv0,i wasnt necessarily talking about people who s...,0


from matplotlib import pyplot as plt
merged_data['Label'].plot(kind='hist', bins=20, title='Label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
merged_data['Label'].plot(kind='line', figsize=(8, 4), title='Label')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# prompt: I want to know how many 1 do I have for the label?

print(merged_data['Label'].value_counts()[1])


634


In [ ]:
merged_data.to_csv('/content/prediction_data_results_withtweet.csv', sep=',', index=False)